<a href="https://colab.research.google.com/github/johncoogan53/XAI_HW4/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #04

Description: This notebook will cover some interpretable methods for machine learning with the imodels python library

### John Coogan
